In [1]:
from torchvision.models import vgg16,VGG16_Weights
import torch.nn as nn
model=vgg16(weights=VGG16_Weights.IMAGENET1K_V1 )

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\KDT/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100%|██████████| 528M/528M [00:15<00:00, 35.0MB/s] 


In [2]:
model.classifier[6]=nn.Linear(4096,2)
for x,y in model.named_parameters():
    for epoch in range(5):
        if 'feature' or (f'classifier' in x and not 'classifier.6.' in x):
            y.requires_grad=False
        if  'classifier.6.' in x:
            y.requires_grad=True

In [9]:
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import PIL as pil
import torch.nn as nn
from torch.utils.data import DataLoader
from torchmetrics.classification import Accuracy 
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau 
import torch
BATCH=1000
EPOCH=100
DEVICE='cuda'if torch.cuda.is_available() else 'cpu'
transform=transforms.Compose([transforms.Resize((256,256),interpolation=transforms.InterpolationMode.BILINEAR),transforms.ToTensor(),transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))])
train_dataset=ImageFolder(r'C:\Users\KDT\Desktop\project\data\image\pet\train',transform)
test_dataset=ImageFolder(r'C:\Users\KDT\Desktop\project\data\image\pet\test',transform)
train_dataloader=DataLoader(train_dataset,BATCH,shuffle=True,drop_last=True)
test_dataloader=DataLoader(test_dataset,BATCH,shuffle=True,drop_last=True)

In [12]:
acurracy=Accuracy('multiclass',num_classes=2)
optimizer=optim.Adam(model.parameters(),lr=0.001)
LR=ReduceLROnPlateau(optimizer,patience=10)
model.to(DEVICE)
for epo in range(EPOCH):
    model.train()
    fact_list=[]
    pred_list=[]
    loss_list=[]
    for x,y in train_dataloader:
        x,y=x.to(DEVICE),y.to(DEVICE)
        optimizer.zero_grad()
        output=model(x)
        loss=nn.functional.cross_entropy(output,y)
        loss.backward()
        optimizer.step()
        pred=torch.argmax(output,dim=1).tolist()
        loss_list.append(loss)
        pred_list.extend(pred)
        fact_list.extend(y.tolist())
    accur=acurracy(torch.LongTensor(pred_list),torch.LongTensor(fact_list))
    print(f'epoch: {epo}')
    print(f'[train] accuracy: {accur}, loss: {sum(loss_list)/len(loss_list)}')
    with torch.no_grad():
        model.eval()
        fact_list=[]
        pred_list=[]
        loss_list=[]
        for x,y in test_dataloader:
            x,y=x.to(DEVICE),y.to(DEVICE)
            output=model(x)
            loss=nn.functional.cross_entropy(output,y).item()
            pred=torch.argmax(output,dim=1).tolist()
            loss_list.append(loss)
            pred_list.extend(pred)
            fact_list.extend(y.tolist())
        accur=acurracy(torch.LongTensor(pred_list),torch.LongTensor(fact_list))
        print(f'epoch: {epo}')
        print(f'[test] accuracy: {accur}, loss: {sum(loss_list)/len(loss_list)}')

epoch: 0
[train] accuracy: 0.9879999756813049, loss: 0.0309020783752203
epoch: 0
[test] accuracy: 0.9890000224113464, loss: 0.03248553071171045
epoch: 1
[train] accuracy: 0.9900000095367432, loss: 0.02750379592180252
epoch: 1
[test] accuracy: 0.9894999861717224, loss: 0.03376724012196064
epoch: 2
[train] accuracy: 0.9902499914169312, loss: 0.02632351778447628
epoch: 2
[test] accuracy: 0.9894999861717224, loss: 0.03287290036678314
epoch: 3
[train] accuracy: 0.9912499785423279, loss: 0.023435313254594803
epoch: 3
[test] accuracy: 0.9900000095367432, loss: 0.03277926333248615
epoch: 4
[train] accuracy: 0.9921249747276306, loss: 0.021511338651180267
epoch: 4
[test] accuracy: 0.9900000095367432, loss: 0.03247069660574198
epoch: 5
[train] accuracy: 0.9919999837875366, loss: 0.021151244640350342
epoch: 5
[test] accuracy: 0.9900000095367432, loss: 0.032306584529578686
epoch: 6
[train] accuracy: 0.9925000071525574, loss: 0.020557686686515808
epoch: 6
[test] accuracy: 0.9900000095367432, loss: 0